</br></br></br></br></br></br>
<h1 style="font-size:10vw"><strong><center>Ah-Jji?</center></strong></h1>
</br></br></br></br></br></br>

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm



import catboost
import xgboost
import lightgbm
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix
from tpot import TPOTClassifier

/home/notitle/anaconda3/envs/loan/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/notitle/anaconda3/envs/loan/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
df = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_submit = pd.read_csv("./data/sample_submission.csv")
df_train = df
df

,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,acc_now_delinq,...,term1,open_acc,installment,revol_util,out_prncp,out_prncp_inv,total_rec_int,fico_range_low,fico_range_high,depvar
0,0.0824,21000.0,29.19,0,1,0,3016,26,0,0,...,1,18,37.74,0.076,0.0,0.0,157.94,765,769,0
1,0.1299,80000.0,4.82,0,1,1,5722,24,0,0,...,1,8,269.52,0.447,0.0,0.0,1702.42,665,669,0
2,0.1299,38000.0,23.66,0,3,0,6511,18,0,0,...,1,7,168.45,0.880,0.0,0.0,1066.64,670,674,0
3,0.1367,100000.0,16.27,4,2,0,6849,30,0,0,...,1,12,510.27,0.457,0.0,0.0,1256.24,680,684,1
4,0.1269,30000.0,25.28,0,1,2,8197,12,0,0,...,1,8,335.45,0.416,0.0,0.0,871.04,660,664,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.1757,65000.0,17.67,0,3,1,11255,21,1,0,...,1,13,718.75,0.780,0.0,0.0,5373.29,660,664,1
99996,0.0890,65000.0,2.88,0,0,0,2105,12,0,0,...,1,7,190.52,0.120,0.0,0.0,835.66,765,769,0
99997,0.1349,46000.0,32.12,0,1,0,8998,20,0,0,...,1,19,217.16,0.643,0.0,0.0,1261.67,665,669,0
99998,0.2115,31000.0,4.53,0,1,0,3875,4,0,0,...,1,3,207.64,0.731,0.0,0.0,1357.69,710,714,1


data preprocessing은 할 필요가 없습니다.   
전부다 되어 있습니다.

In [33]:
# pd.options.display.max_rows = 80

In [1]:
# df_train.nunique()

In [2]:
# df_test.nunique()

In [3]:
X = df_train.drop("depvar", axis=1)
y = df_train["depvar"]

In [4]:
print(len(df_train.columns))
df_train.columns

76


Index(['int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'pub_rec', 'revol_bal', 'total_acc', 'collections_12_mths_ex_med',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens', 'emp_length1',
       'emp_length2', 'emp_length3', 'emp_length4', 'emp_length5',
       'emp_length6', 'emp_length7', 'emp_length8', 'emp_length9',
       'emp_length10', 'emp_length11', 'emp_length12', 'home_ownership1',
       'home_ownership2', 'home_ownership3', 'home_ownership4',
       'home_ownership5', 'home_ownership6', 'verification_status1',
       'verification_status2', 'verification_status3', 'purpose1', 'purpose2',
       'purpose3', 'purpose4', 'purpose5', 'purpose6', 'purpose7', 'purpose8',
       'purpose9', 'purpose10', 'purpose11', 'purpose12', 'purpose13',
       'purpose14', 'initial_list_status1', 'initial_list_status2',
       'mths_since_last_delinq1', 'mths_since_last_delinq2',
       'mths_since_l

In [5]:
df_train["emp_length"] = 0
emp_df = df_train.iloc[:, 15:27]

df_train["purpose"] = 0
pur_df = df_train.iloc[:, 36:50]

df_train["mths_since_last_delinq"] = 0
mths_df = df_train.iloc[:, 52:63]

df_train["home_ownership"] = 0
home_df = df_train.iloc[:, 27:33]

In [6]:
temp = list()
for i in tqdm(range(0,100000)):
  str1 = "emp_length"
  for j in range(1, 13):
    str2 = str(j)
    if emp_df[str1+str2][i] == 1:
      temp.append(j)
      break

df_train["emp_length"] = temp

temp = list()
for i in tqdm(range(0,100000)):
  str1 = "purpose"
  for j in range(1, 15):
    str2 = str(j)
    if pur_df[str1+str2][i] == 1:
      temp.append(j)
      break

df_train["purpose"] = temp

temp = list()
for i in tqdm(range(0,100000)):
  str1 = "mths_since_last_delinq"
  for j in range(1, 12):
    str2 = str(j)
    if mths_df[str1+str2][i] == 1:
      temp.append(j)
      break

df_train["mths_since_last_delinq"] = temp

temp = list()
for i in tqdm(range(0,100000)):
  str1 = "home_ownership"
  for j in range(1, 7):
    str2 = str(j)
    if home_df[str1+str2][i] == 1:
      temp.append(j)
      break

df_train["home_ownership"] = temp


100%|██████████| 100000/100000 [00:03<00:00, 26846.15it/s]


In [7]:
df_train = df_train.drop(columns=[
  'emp_length1', 'emp_length2', 'emp_length3', 'emp_length4',
  'emp_length5', 'emp_length6', 'emp_length7', 'emp_length8',
  'emp_length9', 'emp_length10', 'emp_length11', 'emp_length12',
  'purpose1', 'purpose2', 'purpose3', 'purpose4', 
  'purpose5', 'purpose6', 'purpose7',
  'purpose8', 'purpose9', 'purpose10', 'purpose11', 'purpose12',
  'purpose13', 'purpose14',
  'mths_since_last_delinq1','mths_since_last_delinq2',
  'mths_since_last_delinq3', 'mths_since_last_delinq4',
  'mths_since_last_delinq5', 'mths_since_last_delinq6',
  'mths_since_last_delinq7', 'mths_since_last_delinq8',
  'mths_since_last_delinq9', 'mths_since_last_delinq10',
  'mths_since_last_delinq11',
  'home_ownership1', 'home_ownership2', 'home_ownership3',
  'home_ownership4', 'home_ownership5', 'home_ownership6',
  ])

In [102]:
# 선택사항
# df_train = df_train.drop(columns=[
#   "initial_list_status2", "funded_amnt", "funded_amnt_inv", "out_prncp", "fico_range_high"
#   ])

In [8]:
df_train

,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,acc_now_delinq,...,out_prncp,out_prncp_inv,total_rec_int,fico_range_low,fico_range_high,depvar,emp_length,purpose,mths_since_last_delinq,home_ownership
0,0.0824,21000.0,29.19,0,1,0,3016,26,0,0,...,0.0,0.0,157.94,765,769,0,1,2,1,6
1,0.1299,80000.0,4.82,0,1,1,5722,24,0,0,...,0.0,0.0,1702.42,665,669,0,2,3,1,6
2,0.1299,38000.0,23.66,0,3,0,6511,18,0,0,...,0.0,0.0,1066.64,670,674,0,4,3,11,6
3,0.1367,100000.0,16.27,4,2,0,6849,30,0,0,...,0.0,0.0,1256.24,680,684,1,3,3,5,2
4,0.1269,30000.0,25.28,0,1,2,8197,12,0,0,...,0.0,0.0,871.04,660,664,1,4,3,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.1757,65000.0,17.67,0,3,1,11255,21,1,0,...,0.0,0.0,5373.29,660,664,1,7,3,1,6
99996,0.0890,65000.0,2.88,0,0,0,2105,12,0,0,...,0.0,0.0,835.66,765,769,0,11,3,1,6
99997,0.1349,46000.0,32.12,0,1,0,8998,20,0,0,...,0.0,0.0,1261.67,665,669,0,4,2,1,6
99998,0.2115,31000.0,4.53,0,1,0,3875,4,0,0,...,0.0,0.0,1357.69,710,714,1,12,10,1,6


In [9]:
X_train1 = X[:20000]
X_train2 = X[20000:40000]
X_train3 = X[40000:60000]
X_train4 = X[60000:80000]
X_train5 = X[80000:]

y_train1 = y[:20000]
y_train2 = y[20000:40000]
y_train3 = y[40000:60000]
y_train4 = y[60000:80000]
y_train5 = y[80000:]

In [10]:
##########
frames = [X_train1, X_train2, X_train3, X_train4]
X_train_dataset1 = pd.concat(frames)

frames = [X_train1, X_train2, X_train3, X_train5]
X_train_dataset2 = pd.concat(frames)

frames = [X_train1, X_train2, X_train4, X_train5]
X_train_dataset3 = pd.concat(frames)

frames = [X_train1, X_train3, X_train4, X_train5]
X_train_dataset4 = pd.concat(frames)

frames = [X_train2, X_train3, X_train4, X_train5]
X_train_dataset5 = pd.concat(frames)
##########

frames = [y_train1, y_train2, y_train3, y_train4]
y_train_dataset1 = pd.concat(frames)

frames = [y_train1, y_train2, y_train3, y_train5]
y_train_dataset2 = pd.concat(frames)

frames = [y_train1, y_train2, y_train4, y_train5]
y_train_dataset3 = pd.concat(frames)

frames = [y_train1, y_train3, y_train4, y_train5]
y_train_dataset4 = pd.concat(frames)

frames = [y_train2, y_train3, y_train4, y_train5]
y_train_dataset5 = pd.concat(frames)


In [11]:
model1 = xgboost.XGBClassifier()
model2 = xgboost.XGBClassifier()
model3 = xgboost.XGBClassifier()
model4 = xgboost.XGBClassifier()
model5 = xgboost.XGBClassifier()

model1.fit(X_train_dataset1, y_train_dataset1)
model2.fit(X_train_dataset2, y_train_dataset2)
model3.fit(X_train_dataset3, y_train_dataset3)
model4.fit(X_train_dataset4, y_train_dataset4)
model5.fit(X_train_dataset5, y_train_dataset5)

/home/notitle/anaconda3/envs/loan/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/notitle/anaconda3/envs/loan/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[10:54:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:55:04] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:55:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:55:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
model6 = lightgbm.LGBMClassifier()
model7 = lightgbm.LGBMClassifier()
model8 = lightgbm.LGBMClassifier()
model9 = lightgbm.LGBMClassifier()
model10 = lightgbm.LGBMClassifier()

model6.fit(X_train_dataset1, y_train_dataset1)
model7.fit(X_train_dataset2, y_train_dataset2)
model8.fit(X_train_dataset3, y_train_dataset3)
model9.fit(X_train_dataset4, y_train_dataset4)
model10.fit(X_train_dataset5, y_train_dataset5)

LGBMClassifier()

In [13]:
model11 = catboost.CatBoostClassifier()
model12 = catboost.CatBoostClassifier()
model13 = catboost.CatBoostClassifier()
model14 = catboost.CatBoostClassifier()
model15 = catboost.CatBoostClassifier()

model11.fit(X_train_dataset1, y_train_dataset1)
model12.fit(X_train_dataset2, y_train_dataset2)
model13.fit(X_train_dataset3, y_train_dataset3)
model14.fit(X_train_dataset4, y_train_dataset4)
model15.fit(X_train_dataset5, y_train_dataset5)

Learning rate set to 0.06692
0:	learn: 0.6759286	total: 66.1ms	remaining: 1m 6s
1:	learn: 0.6610865	total: 77.6ms	remaining: 38.7s
2:	learn: 0.6486346	total: 88.8ms	remaining: 29.5s
3:	learn: 0.6368109	total: 99.2ms	remaining: 24.7s
4:	learn: 0.6277035	total: 108ms	remaining: 21.6s
5:	learn: 0.6184388	total: 120ms	remaining: 19.8s
6:	learn: 0.6111359	total: 130ms	remaining: 18.5s
7:	learn: 0.6042963	total: 150ms	remaining: 18.7s
8:	learn: 0.5985637	total: 163ms	remaining: 18s
9:	learn: 0.5937757	total: 177ms	remaining: 17.5s
10:	learn: 0.5902276	total: 218ms	remaining: 19.6s
11:	learn: 0.5860922	total: 260ms	remaining: 21.4s
12:	learn: 0.5822683	total: 293ms	remaining: 22.3s
13:	learn: 0.5788672	total: 325ms	remaining: 22.9s
14:	learn: 0.5757844	total: 364ms	remaining: 23.9s
15:	learn: 0.5737619	total: 403ms	remaining: 24.8s
16:	learn: 0.5716182	total: 449ms	remaining: 25.9s
17:	learn: 0.5692171	total: 473ms	remaining: 25.8s
18:	learn: 0.5671797	total: 503ms	remaining: 26s
19:	learn: 0

In [14]:
y_prob1 = model1.predict_proba(X_train5)
y_prob2 = model2.predict_proba(X_train4)
y_prob3 = model3.predict_proba(X_train3)
y_prob4 = model4.predict_proba(X_train2)
y_prob5 = model5.predict_proba(X_train1)

y_prob6 = model6.predict_proba(X_train5)
y_prob7 = model7.predict_proba(X_train4)
y_prob8 = model8.predict_proba(X_train3)
y_prob9 = model9.predict_proba(X_train2)
y_prob10 = model10.predict_proba(X_train1)

y_prob11 = model11.predict_proba(X_train5)
y_prob12 = model12.predict_proba(X_train4)
y_prob13 = model13.predict_proba(X_train3)
y_prob14 = model14.predict_proba(X_train2)
y_prob15 = model15.predict_proba(X_train1)


In [15]:
# thr 0.3보다 큰 경우
thr = 0.35
print(f"thr > {thr}")
pred1 = (y_prob1[:,1] >= thr).astype(np.int64)
pred2 = (y_prob2[:,1] >= thr).astype(np.int64)
pred3 = (y_prob3[:,1] >= thr).astype(np.int64)
pred4 = (y_prob4[:,1] >= thr).astype(np.int64)
pred5 = (y_prob5[:,1] >= thr).astype(np.int64)
pred6 = (y_prob6[:,1] >= thr).astype(np.int64)
pred7 = (y_prob7[:,1] >= thr).astype(np.int64)
pred8 = (y_prob8[:,1] >= thr).astype(np.int64)
pred9 = (y_prob9[:,1] >= thr).astype(np.int64)
pred10 = (y_prob10[:,1] >= thr).astype(np.int64)
pred11 = (y_prob11[:,1] >= thr).astype(np.int64)
pred12 = (y_prob12[:,1] >= thr).astype(np.int64)
pred13 = (y_prob13[:,1] >= thr).astype(np.int64)
pred14 = (y_prob14[:,1] >= thr).astype(np.int64)
pred15 = (y_prob15[:,1] >= thr).astype(np.int64)

print(f" model1의 예측비율 : {pred1.sum() / len(pred1)}")
print(f" model2의 예측비율 : {pred2.sum() / len(pred2)}")
print(f" model3의 예측비율 : {pred3.sum() / len(pred3)}")
print(f" model4의 예측비율 : {pred4.sum() / len(pred4)}")
print(f" model5의 예측비율 : {pred5.sum() / len(pred5)}")
print(f" model6의 예측비율 : {pred6.sum() / len(pred6)}")
print(f" model7의 예측비율 : {pred7.sum() / len(pred7)}")
print(f" model8의 예측비율 : {pred8.sum() / len(pred8)}")
print(f" model9의 예측비율 : {pred9.sum() / len(pred9)}")
print(f" model10의 예측비율 : {pred10.sum() / len(pred10)}")
print(f" model11의 예측비율 : {pred11.sum() / len(pred11)}")
print(f" model12의 예측비율 : {pred12.sum() / len(pred12)}")
print(f" model13의 예측비율 : {pred13.sum() / len(pred13)}")
print(f" model14의 예측비율 : {pred14.sum() / len(pred14)}")
print(f" model15의 예측비율 : {pred15.sum() / len(pred15)}")

thr > 0.35
 model1의 예측비율 : 0.37745
 model2의 예측비율 : 0.38155
 model3의 예측비율 : 0.3768
 model4의 예측비율 : 0.3799
 model5의 예측비율 : 0.37465
 model6의 예측비율 : 0.37305
 model7의 예측비율 : 0.3825
 model8의 예측비율 : 0.3756
 model9의 예측비율 : 0.3755
 model10의 예측비율 : 0.3704
 model11의 예측비율 : 0.376
 model12의 예측비율 : 0.38415
 model13의 예측비율 : 0.37595
 model14의 예측비율 : 0.37735
 model15의 예측비율 : 0.37725


In [16]:
def get_clf_eval(y_actual, y_pred, model):
    # accuracy = accuracy_score(y_actual, y_pred)
    # precision = precision_score(y_actual, y_pred)
    # recall = recall_score(y_actual, y_pred)
    AUC = roc_auc_score(y_actual, y_pred)
    F1 = f1_score(y_actual, y_pred)
    # print('\n정확도: {:.4f}'.format(accuracy))
    # print('정밀도: {:.4f}'.format(precision))
    # print('재현율: {:.4f}'.format(recall))
    print(model)
    print('AUC: {:.4f}'.format(AUC))
    print('F1: {:.4f}'.format(F1))
    print()

In [17]:
# xgboost 성능 확인
get_clf_eval(y_train5, pred1, "model 1")
get_clf_eval(y_train4, pred2, "model 2")
get_clf_eval(y_train3, pred3, "model 3")
get_clf_eval(y_train2, pred4, "model 4")
get_clf_eval(y_train1, pred5, "model 5")


model 1
AUC: 0.7240
F1: 0.6246

model 2
AUC: 0.7208
F1: 0.6301

model 3
AUC: 0.7230
F1: 0.6305

model 4
AUC: 0.7215
F1: 0.6245

model 5
AUC: 0.7156
F1: 0.6188



In [18]:
get_clf_eval(y_train5, pred6, "model 6")
get_clf_eval(y_train4, pred7, "model 7")
get_clf_eval(y_train3, pred8, "model 8")
get_clf_eval(y_train2, pred9, "model 9")
get_clf_eval(y_train1, pred10, "model 10")

model 6
AUC: 0.7262
F1: 0.6273

model 7
AUC: 0.7223
F1: 0.6320

model 8
AUC: 0.7249
F1: 0.6329

model 9
AUC: 0.7244
F1: 0.6280

model 10
AUC: 0.7223
F1: 0.6272



In [19]:
get_clf_eval(y_train5, pred11, "model 11")
get_clf_eval(y_train4, pred12, "model 12")
get_clf_eval(y_train3, pred13, "model 13")
get_clf_eval(y_train2, pred14, "model 14")
get_clf_eval(y_train1, pred15, "model 15")

model 11
AUC: 0.7296
F1: 0.6317

model 12
AUC: 0.7275
F1: 0.6386

model 13
AUC: 0.7277
F1: 0.6364

model 14
AUC: 0.7266
F1: 0.6308

model 15
AUC: 0.7257
F1: 0.6316



In [20]:
# 제출 양식 다운로드
submit = pd.read_csv('./data/sample_submission.csv')

# prediction 수행
df_test = pd.read_csv('./data/test.csv')

In [21]:
test = df_test.drop('ID', axis=1)

In [38]:
prob1 = model1.predict_proba(test)
out1 = (prob1[:,1] >= thr).astype(np.int64)
prob2 = model2.predict_proba(test)
out2 = (prob2[:,1] >= thr).astype(np.int64)
prob3 = model3.predict_proba(test)
out3 = (prob3[:,1] >= thr).astype(np.int64)
prob4 = model4.predict_proba(test)
out4 = (prob4[:,1] >= thr).astype(np.int64)
prob5 = model5.predict_proba(test)
out5 = (prob5[:,1] >= thr).astype(np.int64)

prob6 = model6.predict_proba(test)
out6 = (prob6[:,1] >= thr).astype(np.int64)
prob7 = model7.predict_proba(test)
out7 = (prob7[:,1] >= thr).astype(np.int64)
prob8 = model8.predict_proba(test)
out8 = (prob8[:,1] >= thr).astype(np.int64)
prob9 = model9.predict_proba(test)
out9 = (prob9[:,1] >= thr).astype(np.int64)
prob10 = model10.predict_proba(test)
out10 = (prob10[:,1] >= thr).astype(np.int64)

prob11 = model11.predict_proba(test)
out11 = (prob11[:,1] >= thr).astype(np.int64)
prob12 = model12.predict_proba(test)
out12 = (prob12[:,1] >= thr).astype(np.int64)
prob13 = model13.predict_proba(test)
out13 = (prob13[:,1] >= thr).astype(np.int64)
prob14 = model14.predict_proba(test)
out14 = (prob14[:,1] >= thr).astype(np.int64)
prob15 = model15.predict_proba(test)
out15 = (prob15[:,1] >= thr).astype(np.int64)





print(out1.sum() / len(out1))
print(out2.sum() / len(out2))
print(out3.sum() / len(out3))
print(out4.sum() / len(out4))
print(out5.sum() / len(out5))
print(out6.sum() / len(out6))
print(out7.sum() / len(out7))
print(out8.sum() / len(out8))
print(out9.sum() / len(out9))
print(out10.sum() / len(out10))
print(out11.sum() / len(out11))
print(out12.sum() / len(out12))
print(out13.sum() / len(out13))
print(out14.sum() / len(out14))
print(out15.sum() / len(out15))







0.3856097833370561
0.3853305785123967
0.385637703819522
0.38658700022336384
0.3859169086441814
0.38128210855483585
0.38172883627429083
0.381198347107438
0.38175675675675674
0.378992628992629
0.3858052267143176
0.3860285905740451
0.3832365423274514
0.3866428411882957
0.38418583873129325


In [68]:
result = []
for i in range(len(test)):
  temp = 0
  for j in range(out_list.shape[0]):
    temp += out_list[j][i]
  if 6 <= temp:
    result.append(1)
  else:
    result.append(0)

In [69]:
print(f" 최종 결과물!!! ")
print(f" {sum(result) / len(result)} ")
print(len(result))
print(result.count(1))

submit["answer"] = result  

 최종 결과물!!! 
 0.40037971856153676 
35816
14340


In [ ]:
# 제출 파일 저장
submit.to_csv('./submit/submit.csv', index=False)